## データについて
各カラムの意味は以下の通りです.
<table>
  <thead>
  <tr>
    <td>.csvファイル</td>
    <td>学習モデル</td>
    <td>CVスコア</td>
    <td>暫定スコア</td>
    <td>備考</td>     
  </tr>
  </thead>
  <tr>
    <td>submission06102200.csv</td>
    <td>LR only</td> 
    <td>0.87363</td>
    <td>0.86669</td>
    <td>officer_idは100000のみ</td>
  </tr>
  <tr>
    <td>submission06111044.csv</td>
    <td>{LR & RF(n=100)}/2</td> 
    <td>0.87809</td>
    <td>0.?????</td>
    <td>同上</td>
  </tr>
  <tr>
    <td>submission06111100.csv</td>
    <td>{LR & RF(n=100)}/2</td> 
    <td>0.88333</td>
    <td>0.?????</td>
    <td>stop_hourを修正</td>
  </tr> 
  <tr>
    <td>なし</td>
    <td>XGB(深さ5)</td> 
    <td>0.88728</td>
    <td>0.?????</td>
    <td></td>
  </tr>  
  <tr>
    <td>なし</td>
    <td>XGB(深さ4,n_est=200)</td> 
    <td>0.88848</td>
    <td>0.?????</td>
    <td></td>
  </tr>   
  <tr>
    <td>なし</td>
    <td>XGB(深さ5)とLRを10:1</td> 
    <td>0.88766</td>
    <td>0.?????</td>
    <td></td>
  </tr>  
  <tr>
    <td>なし</td>
    <td>XGB(深さ4)</td> 
    <td>0.86897</td>
    <td>0.?????</td>
    <td>officer_idを追加した</td>
  </tr>  
  <tr>
    <td>submission_06111430.csv</td>
    <td>XGB(確か4と400)</td> 
    <td>0.?????</td>
    <td>0.?????</td>
    <td>nyx1で実行</td>
  </tr>   
  <tr>
    <td>submission_06111430.csv</td>
    <td>XGB&LR&RF</td> 
    <td>0.88907</td>
    <td>0.?????</td>
    <td>nyx1で実行</td>
  </tr>  
  <tr>
    <td>submission_06131630.csv</td>
    <td>xgb(4,200):mlp:lgb=70:70:3</td> 
    <td>0.8869</td>
    <td>0.?????</td>
    <td>nyx1で実行、_clean_06111230を使用</td>
  </tr>   
  <tr>
    <td>submission_06131630.csv</td>
    <td>xgb(4,200):mlp=1:1</td> 
    <td>0.8892</td>
    <td>0.?????</td>
    <td>nyx1で実行、_clean_06131800を使用。lgbmいらんかった...。</td>
  </tr>     
</table>


In [51]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from datetime import datetime

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
#m = xgb.XGBRegressor()
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMClassifier
from sklearn.grid_search import GridSearchCV

In [2]:
df = pd.read_csv("train_clean_06141200.csv")
df_test = pd.read_csv("test_clean_06141200.csv")

In [52]:
df = pd.read_csv("train_clean_06142000.csv")
df_test = pd.read_csv("test_clean_06142000.csv")

In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219319 entries, 0 to 219318
Columns: 267 entries, driver_gender to location_I395
dtypes: float64(192), int64(75)
memory usage: 446.8 MB


In [68]:
sc = StandardScaler()
df[["driver_age_raw","population","area","stop_day","X","Y"]]= sc.fit_transform(df[["driver_age_raw","population","area","stop_day","X","Y"]])
df_test[["driver_age_raw","population","area","stop_day","X","Y"]] = sc.fit_transform(df_test[["driver_age_raw","population","area","stop_day","X","Y"]])

In [55]:
X = df.drop(["is_arrested"],axis=1)
y = df["is_arrested"]

In [75]:
X = df.drop(["is_arrested",'location_monroe', 'location_north branford', 'location_avon',
       'location_portland', 'location_easton', 'location_bloomfield',
       'location_ridgefield', 'location_wolcott', 'location_simsbury',
       'location_granby', 'location_wilton', 'officer_55887736',
       'officer_770626977', 'officer_1000002961', 'officer_1000002376',
       'officer_70583331', 'officer_1000002714', 'officer_1000003026',
       'officer_1000002219', 'officer_298463910', 'officer_1000002220',
       'officer_1000003191'],axis=1)
y = df["is_arrested"]

In [76]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

/home/minoda/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [ ]:
param_grid = {"max_depth": [3,4],
              "n_estimators":[50,100,300,500],
              #"reg_lambda": [1, 100, 500],
              #"num_leaves": [20, 30, 40],
              "learning_rate": [0.01, 0.1, 1]
             }
lgbm_grid = GridSearchCV(estimator=LGBMClassifier(random_state=0),
                 param_grid = param_grid,   
                 scoring="roc_auc", 
                 cv = 10,
                 n_jobs = 8)   
lgbm_grid.fit(X,y) 
lgbm_grid_best = lgbm_grid.best_estimator_
print("Best Model Parameter: ",lgbm_grid.best_params_)

In [82]:
%%time
lgbc = LGBMClassifier(class_weight="balanced",n_estimators=30,learning_rate=0.1
                      ,reg_lambda=4,num_leaves=25)
lgbc.fit(X_train,y_train)
y_predict_lgb = lgbc.predict_proba(X_test)[:,1]
y_train_predict_lgb = lgbc.predict_proba(X_train)[:,1]
print("train data score:",roc_auc_score(y_train.values,y_train_predict_lgb))
print("cross validation score:",roc_auc_score(y_test.values,y_predict_lgb))

train data score: 0.8964912953361709
cross validation score: 0.8741031798441689
CPU times: user 3.16 s, sys: 728 ms, total: 3.89 s
Wall time: 3.89 s


In [59]:
%%time
xgbc = xgb.XGBClassifier(max_depth=3,n_estimators=500)
xgbc.fit(X_train,y_train)
y_predict_xgb = xgbc.predict_proba(X_test)[:,1]
y_train_predict_xgb = xgbc.predict_proba(X_train)[:,1]
print("train data score:",roc_auc_score(y_train.values,y_train_predict_xgb))
print("cross validation score:",roc_auc_score(y_test.values,y_predict_xgb))

train data score: 0.905948340656719
cross validation score: 0.8691488779725417
CPU times: user 6min 24s, sys: 1min 39s, total: 8min 4s
Wall time: 8min 2s


<table class="table2">
    <caption>06121820</caption>
  <tr><th></th><th>3</th><th>4</th><th>5</th><th>6</th></tr>
  <tr><td>100</td><td>???,???</td><td>??? / ???</td><td>0.8990 / 0.8768</td><td>???,???</td></tr>
  <tr><td>200</td><td>???,???</td><td>??? / </td><td>???,???</td><td>???,???</td></tr>
  <tr><td>300</td><td>???,???</td><td>??? / >???</td><td>???,???</td><td>???,???</td></tr>
    <tr><td>500</td><td>???? / ????</td><td>0.9181 / 0.8734</td><td>?? / ???</td><td>???,???</td></tr>
  <tr><td>700</td><td>0.9054 / 0.8726</td><td>???? / ????</td><td>??? / ???</td><td>???,???</td>
</table>


<table class="table2">
    <caption>06111230</caption>
  <tr><th></th><th>3</th><th>4</th><th>5</th><th>6</th></tr>
  <tr><td>100</td><td>???,???</td><td>0.8903 / 0.8768</td><td>??? / 0.8872?</td><td>???,???</td></tr>
  <tr><td>200</td><td>???,???</td><td>??? / </td><td>???,???</td><td>???,???</td></tr>
  <tr><td>300</td><td>???,???</td><td>0.9093 / 0.8793</td><td>???,???</td><td>???,???</td></tr>
    <tr><td>500</td><td>0.9004,0.8791</td><td>0.9184 / 0.8811</td><td>0.9376 / 0.8699</td><td>???,???</td></tr>
  <tr><td>700</td><td>???,???</td><td>0.9272 / 0.8809</td><td>??? / ???</td><td>???,???</td>
</table>

In [83]:
%%time
mlp = MLPClassifier(alpha=1e-2,hidden_layer_sizes=6, random_state=1)
mlp.fit(X_train,y_train)
y_predict_mlp = mlp.predict_proba(X_test)[:,1]
y_train_predict_mlp = mlp.predict_proba(X_train)[:,1]
print("train data score:",roc_auc_score(y_train.values,y_train_predict_mlp))
print("cross validation score:",roc_auc_score(y_test.values,y_predict_mlp))

train data score: 0.8916114162510637
cross validation score: 0.8734176930019586
CPU times: user 6.78 s, sys: 1.31 s, total: 8.09 s
Wall time: 8.08 s


In [72]:
%%time
lr = LogisticRegression(class_weight="balanced",C=0.1)
lr.fit(X_train,y_train)
y_predict_lr = lr.predict_proba(X_test)[:,1]
y_train_predict_lr = lr.predict_proba(X_train)[:,1]
print("train data score:",roc_auc_score(y_train.values,y_train_predict_lr))
print("cross validation score:",roc_auc_score(y_test.values,y_predict_lr))

train data score: 0.8808309376907693
cross validation score: 0.8696708108494071
CPU times: user 3.04 s, sys: 228 ms, total: 3.27 s
Wall time: 3.27 s


In [33]:
%%time
forest = RandomForestClassifier(max_depth=3,n_estimators=200,class_weight="balanced")
forest.fit(X_train,y_train)
y_predict_forest = forest.predict_proba(X_test)[:,1]
y_train_predict_forest = forest.predict_proba(X_train)[:,1]
print("train data score:",roc_auc_score(y_train.values,y_train_predict_forest))
print("cross validation score:",roc_auc_score(y_test.values,y_predict_forest))

train data score: 0.8619618898679375
cross validation score: 0.8436802550260822
CPU times: user 26.7 s, sys: 660 ms, total: 27.4 s
Wall time: 12.2 s


In [44]:
vec = forest.feature_importances_
index=np.where(vec>0.001)

In [45]:
df.columns.values[index]

array(['driver_gender', 'driver_age_raw', 'contraband_found',
       'stop_duration', 'is_arrested', 'violation_Equipment',
       'violation_License', 'violation_Lights',
       'violation_Moving violation', 'violation_Other',
       'violation_Safe movement', 'violation_Seat belt',
       'stop_day_Monday', 'stop_day_Saturday', 'stop_day_Wednesday',
       'stop_hour_0.0', 'stop_hour_1.0', 'stop_hour_2.0', 'stop_hour_5.0',
       'stop_hour_8.0', 'stop_hour_22.0', 'race_Black', 'race_Other',
       'race_White', 'search_Consent', 'search_Inventory',
       'county_Litchfield County', 'county_New London County',
       'county_Windham County', 'location_route', 'location_00000',
       'population', 'area', 'X', 'location_deep river',
       'location_wallingford', 'location_chester', 'location_ansonia',
       'location_branford', 'location_coventry', 'officer_1000003191'],
      dtype=object)

In [120]:
%%time
adbc = AdaBoostClassifier(n_estimators=100)
adbc.fit(X_train,y_train)
y_predict_adb = adbc.predict_proba(X_test)[:,1]
y_train_predict_adb = adbc.predict_proba(X_train)[:,1]
print("train data score:",roc_auc_score(y_train.values,y_train_predict_adb))
print("cross validation score:",roc_auc_score(y_test.values,y_predict_adb))

train data score: 0.87848785683378
cross validation score: 0.8695935235015916
CPU times: user 30.7 s, sys: 208 ms, total: 30.9 s
Wall time: 30.9 s


<table>
  <caption>clean_06121840</caption>
  <thead>
  <tr>
    <td>あるふぁ</td>
    <td>隠れ層サイズ</td>
    <td>train data score</td>  
    <td>cross validation score</td>       
  </tr>
  </thead>
  <tr>
    <td>1e-2</td>
    <td>10</td> 
    <td>0.8877</td>
    <td>0.8893</td>
  </tr>
  <tr>
    <td>1e-2</td>
    <td>6</td> 
    <td>0.8887</td>
    <td>0.8825</td>
  </tr> 
</table>
<table>
  <caption>clean_06111230</caption>
  <thead>
  <tr>
    <td>あるふぁ</td>
    <td>隠れ層サイズ</td>
    <td>train data score</td>  
    <td>cross validation score</td>       
  </tr>
  </thead>
  <tr>
    <td>1e-5</td>
    <td>50</td> 
    <td>0.96</td>
    <td>0.82</td>
  </tr> 
  <tr>
    <td>1e-5</td>
    <td>30</td> 
    <td>0.946</td>
    <td>0.8284</td>
  </tr>    
  <tr>
    <td>1e-5</td>
    <td>20</td> 
    <td>0.9258</td>
    <td>0.8543</td>
  </tr>   
  <tr>
    <td>1e-5</td>
    <td>10</td> 
    <td>0.9072</td>
    <td>0.8718</td>
  </tr> 
  <tr>
    <td>1e-5</td>
    <td>6</td> 
    <td>0.8934</td>
    <td>0.8802</td>
  </tr>   
  <tr>
    <td>1e-3</td>
    <td>6</td> 
    <td>0.8923</td>
    <td>0.8828</td>
  </tr>
  <tr>
    <td>1e-2</td>
    <td>6</td> 
    <td>0.8910</td>
    <td>0.8834</td>
  </tr>   
  <tr>
    <td>1e-5</td>
    <td>3</td> 
    <td>0.8747</td>
    <td>0.8612</td>
  </tr>   
</table>

In [455]:
#lr,xgb,mlp,lgbm
rate = np.array([1,1,0])
roc_auc_score(y_test.values,np.dot(rate,np.array([y_predict_xgb
                                                  ,y_predict_mlp,y_predict_lgb]))/np.sum(rate))

0.8892448665994271

In [467]:
X_final_test = df_test
X_final_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93994 entries, 0 to 93993
Columns: 252 entries, driver_gender to location_I395
dtypes: float64(179), int64(73)
memory usage: 180.7 MB


In [224]:
%%time
lgbc_final = LGBMClassifier(class_weight="balanced",n_estimators=30,learning_rate=0.3
                      ,reg_lambda=40,num_leaves=45)
lgbc_final.fit(X,y)

CPU times: user 2.25 s, sys: 568 ms, total: 2.82 s
Wall time: 2.82 s


In [456]:
%%time
xgb_final = xgb.XGBClassifier(max_depth=4,n_estimators=200)
xgb_final.fit(X,y)

CPU times: user 4min 4s, sys: 1min 1s, total: 5min 6s
Wall time: 5min 8s


In [457]:
%%time
mlp_final = MLPClassifier(alpha=1e-2,hidden_layer_sizes=6, random_state=1)
mlp_final.fit(X,y)

CPU times: user 11.2 s, sys: 1.78 s, total: 13 s
Wall time: 13.1 s


In [ ]:
y_final_predict_xgb = xgb_final.predict_proba(X_final_test)[:,1]
y_final_predict_mlp = mlp_final.predict_proba(X_final_test)[:,1]
rate = np.array([1,1])
y_final_predict = np.dot(rate,np.array([y_final_predict_xgb,y_final_predict_mlp]))/np.sum(rate)

In [235]:
y_final_predict = np.round(y_final_predict,6)

In [236]:
np.average(y_final_predict)

0.029069573430218952

In [237]:
submission = pd.DataFrame(data=y_final_predict, columns=['is_arrested'])
submission.head(3)

,is_arrested
0,0.012128
1,0.024653
2,0.437279


In [238]:
submission.to_csv("submission.csv",index=False)